In [1]:
import re
from datetime import datetime

In [2]:
cur_identifier = None
cur_alignment = None
cur_results = None
cur_state = ""
last_datetime_stored = None
last_start_time = None
all_results = []
with open("../preprocessed/bash_log.txt", "r") as handle:
    for line in handle:
        line = line.rstrip()
        try:
            last_datetime_stored = datetime.strptime(line[:26], '%Y-%m-%d %H:%M:%S.%f')
        except:
            pass
        if cur_state == "":
            match_identifier = re.search("^-+sample-(?P<n>\d+)-(?P<i>\d+)-+$", line)
            if match_identifier:
                cur_identifier = (int(match_identifier.group("n")), int(match_identifier.group("i")))
                cur_state = "parse_alignment"
                cur_alignment = None
                cur_results = {}
        elif cur_state == "parse_alignment":
            m = re.search("^(.+):  Predicting domains for \d+ fasta files$", line)
            if m:
                cur_results["start_hmmscan"] = datetime.strptime(m.group(1), '%Y-%m-%d %H:%M:%S.%f')
                continue
            m = re.search("^(.+):  Finished generating domtable files.$", line)
            if m:
                cur_results["end_hmmscan"] = datetime.strptime(m.group(1), '%Y-%m-%d %H:%M:%S.%f')
                if "start_hmmscan" in cur_results:
                    cur_results["hmmscan"] = (cur_results["end_hmmscan"] - cur_results["start_hmmscan"]).total_seconds()
                continue
            m = re.search("^launch_(.+) took (.+) seconds$", line)
            if m:
                cur_alignment = m.group(1)
                cur_results["alignment"] = float(m.group(2))
                continue
            m = re.search("^\s+Main function took (.+) s$", line)
            if m:
                cur_results["main_function"] = float(m.group(1))
                continue
            if line == "   - - Processing input files - -":
                if len(cur_results.keys()) > 0:
                    runtime = (last_datetime_stored - last_start_time).total_seconds()
                    all_results.append([
                        cur_identifier[0],
                        cur_identifier[1],
                        cur_alignment,
                        cur_results.get("hmmscan"),
                        cur_results.get("alignment"),
                        cur_results.get("main_function"),
                        runtime
                    ])
                    cur_alignment = None
                    cur_results = {}
                last_start_time = last_datetime_stored
                continue
            m = re.search("^-+sample-(\d+)-(\d+)-+$", line)
            if m:
                if len(cur_results.keys()) > 0:
                    runtime = (last_datetime_stored - last_start_time).total_seconds()
                    all_results.append([
                        cur_identifier[0],
                        cur_identifier[1],
                        cur_alignment,
                        cur_results.get("hmmscan"),
                        cur_results.get("alignment"),
                        cur_results.get("main_function"),
                        runtime
                    ])
                    cur_alignment = None
                    cur_results = {}
                cur_state = ""
                continue

In [3]:
all_results

[[10, 1, 'hmmalign', 13.23424, 1.558, 25.487, 29.847174],
 [10, 1, None, 13.232056, None, 24.408, 30.105126],
 [10, 1, 'mafft', 13.032096, 153.693, 177.507, 177.357672],
 [10, 2, None, 14.036431, None, 24.899, 29.251051],
 [10, 2, None, 13.931341, None, 25.156, 30.03802],
 [10, 2, 'mafft', 14.133796, 71.751, 95.563, 95.424423],
 [10, 3, 'hmmalign', 11.225652, 1.456, 24.125, 29.211449],
 [10, 3, None, 11.027263, None, 22.486, 26.838825],
 [10, 3, 'mafft', 10.923958, 126.204, 147.677, 147.522585],
 [10, 4, 'hmmalign', 14.63814, 1.355, 27.471, 31.629525],
 [10, 4, None, 14.613227, None, 27.468, 32.013074],
 [10, 4, 'mafft', 14.93363, 116.605, 146.393, 146.240281],
 [10, 5, 'hmmalign', 12.132277, 1.556, 27.042, 31.717342],
 [10, 5, None, 11.828454, None, 23.652, 28.20774],
 [10, 5, 'mafft', 11.829286, 142.503, 166.001, 165.901993],
 [100, 1, 'hmmalign', 67.072134, 17.801, 133.857, 138.580328],
 [100, 1, 'muscle', 73.803125, 13.595, 141.655, 146.450559],
 [100, 1, 'mafft', 74.814332, 2125.9